In [7]:
import boto3
import json
import base64
import io
from PIL import Image

# Initialize Amazon Bedrock client
bedrock_client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Define the model ID
model_id = "amazon.nova-canvas-v1:0"

# Function to generate an image using Nova Canvas
def generate_image(prompt):
    """Generates an image from a text prompt."""
    payload = json.dumps({
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "height": 1024,
            "width": 1024,
            "cfgScale": 8.0,
            "seed": 42
        }
    })
    
    response = bedrock_client.invoke_model(
        modelId=model_id, body=payload, accept="application/json", contentType="application/json"
    )
    result = json.loads(response["body"].read())
    base64_image = result.get("images")[0]
    
    # Convert base64 to image
    image_bytes = base64.b64decode(base64_image.encode("ascii"))
    return Image.open(io.BytesIO(image_bytes)), base64_image  # Return both image and base64 data

# Function to edit an image using IMAGE_VARIATION
def edit_image(base64_original, edit_prompt, similarity=0.75):
    """Edits an existing image using Nova Canvas IMAGE_VARIATION."""
    payload = json.dumps({
        "taskType": "IMAGE_VARIATION",
        "imageVariationParams": {
            "images": [base64_original],  # Input image(s) as Base64
            "similarityStrength": similarity,  # Controls how much variation is applied
            "text": edit_prompt,  # Instruction for modification
            "negativeText": "blurry, low quality, distorted"  # Prevents unwanted artifacts
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "height": 1024,
            "width": 1024,
            "cfgScale": 7.0
        }
    })

    response = bedrock_client.invoke_model(
        modelId=model_id, body=payload, accept="application/json", contentType="application/json"
    )
    result = json.loads(response["body"].read())
    base64_edited = result.get("images")[0]

    # Convert base64 to edited image
    edited_image_bytes = base64.b64decode(base64_edited.encode("ascii"))
    return Image.open(io.BytesIO(edited_image_bytes))

# Step 1: Generate an image
prompt = "boy standing in park with red tshirt"
original_image, base64_original = generate_image(prompt)
original_image.show()  # Show original image in VS Code

# Step 2: Edit the image using IMAGE_VARIATION
edit_prompt = "Make the color of tshirt blue"
edited_image = edit_image(base64_original, edit_prompt)
edited_image.show()  # Show edited image in VS Code
